In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

# Network Parameters
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

# Store layers weight & bias
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting ./train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting ./train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./t10k-labels-idx1-ubyte.gz


In [2]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

In [3]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='SAME')

In [4]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf. global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: batch_x,
                y: batch_y,
                keep_prob: 1.})
            valid_acc = sess.run(accuracy, feed_dict={
                x: mnist.validation.images[:test_valid_size],
                y: mnist.validation.labels[:test_valid_size],
                keep_prob: 1.})

            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))

    # Calculate Test Accuracy
    test_acc = sess.run(accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))

Epoch  1, Batch   1 -Loss: 64357.8438 Validation Accuracy: 0.132812
Epoch  1, Batch   2 -Loss: 57765.6406 Validation Accuracy: 0.136719
Epoch  1, Batch   3 -Loss: 44333.1914 Validation Accuracy: 0.144531
Epoch  1, Batch   4 -Loss: 39131.3047 Validation Accuracy: 0.164062
Epoch  1, Batch   5 -Loss: 35829.4258 Validation Accuracy: 0.125000
Epoch  1, Batch   6 -Loss: 32560.0098 Validation Accuracy: 0.140625
Epoch  1, Batch   7 -Loss: 29611.0938 Validation Accuracy: 0.144531
Epoch  1, Batch   8 -Loss: 28942.0234 Validation Accuracy: 0.187500
Epoch  1, Batch   9 -Loss: 25708.6055 Validation Accuracy: 0.187500
Epoch  1, Batch  10 -Loss: 25839.9609 Validation Accuracy: 0.183594
Epoch  1, Batch  11 -Loss: 24538.1836 Validation Accuracy: 0.195312
Epoch  1, Batch  12 -Loss: 19993.8926 Validation Accuracy: 0.207031
Epoch  1, Batch  13 -Loss: 20675.7266 Validation Accuracy: 0.210938
Epoch  1, Batch  14 -Loss: 17845.0449 Validation Accuracy: 0.218750
Epoch  1, Batch  15 -Loss: 19462.8867 Validation

Epoch  1, Batch 122 -Loss:  4394.0913 Validation Accuracy: 0.667969
Epoch  1, Batch 123 -Loss:  5191.1025 Validation Accuracy: 0.664062
Epoch  1, Batch 124 -Loss:  3739.1826 Validation Accuracy: 0.660156
Epoch  1, Batch 125 -Loss:  4428.4893 Validation Accuracy: 0.671875
Epoch  1, Batch 126 -Loss:  4439.8496 Validation Accuracy: 0.675781
Epoch  1, Batch 127 -Loss:  4440.4512 Validation Accuracy: 0.671875
Epoch  1, Batch 128 -Loss:  5487.0610 Validation Accuracy: 0.679688
Epoch  1, Batch 129 -Loss:  4755.6499 Validation Accuracy: 0.671875
Epoch  1, Batch 130 -Loss:  3871.6709 Validation Accuracy: 0.671875
Epoch  1, Batch 131 -Loss:  4443.2290 Validation Accuracy: 0.667969
Epoch  1, Batch 132 -Loss:  3154.1016 Validation Accuracy: 0.667969
Epoch  1, Batch 133 -Loss:  4627.5312 Validation Accuracy: 0.660156
Epoch  1, Batch 134 -Loss:  5474.5566 Validation Accuracy: 0.675781
Epoch  1, Batch 135 -Loss:  3861.4434 Validation Accuracy: 0.667969
Epoch  1, Batch 136 -Loss:  4878.5903 Validation

Epoch  1, Batch 243 -Loss:  2577.2993 Validation Accuracy: 0.750000
Epoch  1, Batch 244 -Loss:  3215.4541 Validation Accuracy: 0.753906
Epoch  1, Batch 245 -Loss:  3380.1526 Validation Accuracy: 0.746094
Epoch  1, Batch 246 -Loss:  2094.1836 Validation Accuracy: 0.738281
Epoch  1, Batch 247 -Loss:  2808.3948 Validation Accuracy: 0.738281
Epoch  1, Batch 248 -Loss:  1898.9722 Validation Accuracy: 0.738281
Epoch  1, Batch 249 -Loss:  2523.6091 Validation Accuracy: 0.738281
Epoch  1, Batch 250 -Loss:  2421.3015 Validation Accuracy: 0.738281
Epoch  1, Batch 251 -Loss:  1966.4656 Validation Accuracy: 0.742188
Epoch  1, Batch 252 -Loss:  1841.8352 Validation Accuracy: 0.742188
Epoch  1, Batch 253 -Loss:  2591.8440 Validation Accuracy: 0.750000
Epoch  1, Batch 254 -Loss:  2828.1846 Validation Accuracy: 0.753906
Epoch  1, Batch 255 -Loss:  1844.9951 Validation Accuracy: 0.757812
Epoch  1, Batch 256 -Loss:  2338.9553 Validation Accuracy: 0.753906
Epoch  1, Batch 257 -Loss:  2895.6880 Validation

Epoch  1, Batch 364 -Loss:  1849.4062 Validation Accuracy: 0.769531
Epoch  1, Batch 365 -Loss:  1465.2228 Validation Accuracy: 0.785156
Epoch  1, Batch 366 -Loss:  2221.9434 Validation Accuracy: 0.785156
Epoch  1, Batch 367 -Loss:  1812.5820 Validation Accuracy: 0.781250
Epoch  1, Batch 368 -Loss:  2217.1250 Validation Accuracy: 0.773438
Epoch  1, Batch 369 -Loss:  2367.2654 Validation Accuracy: 0.781250
Epoch  1, Batch 370 -Loss:  1612.3142 Validation Accuracy: 0.781250
Epoch  1, Batch 371 -Loss:  1720.0536 Validation Accuracy: 0.785156
Epoch  1, Batch 372 -Loss:  1650.5956 Validation Accuracy: 0.777344
Epoch  1, Batch 373 -Loss:  2301.4316 Validation Accuracy: 0.773438
Epoch  1, Batch 374 -Loss:  1281.9509 Validation Accuracy: 0.773438
Epoch  1, Batch 375 -Loss:  1343.5104 Validation Accuracy: 0.773438
Epoch  1, Batch 376 -Loss:  1914.1688 Validation Accuracy: 0.769531
Epoch  1, Batch 377 -Loss:  2342.0259 Validation Accuracy: 0.773438
Epoch  1, Batch 378 -Loss:  1828.7271 Validation

Epoch  2, Batch  56 -Loss:  1300.3354 Validation Accuracy: 0.777344
Epoch  2, Batch  57 -Loss:  1632.2673 Validation Accuracy: 0.777344
Epoch  2, Batch  58 -Loss:   976.4775 Validation Accuracy: 0.777344
Epoch  2, Batch  59 -Loss:  1897.9194 Validation Accuracy: 0.773438
Epoch  2, Batch  60 -Loss:  1788.8550 Validation Accuracy: 0.773438
Epoch  2, Batch  61 -Loss:  1488.9825 Validation Accuracy: 0.773438
Epoch  2, Batch  62 -Loss:  1509.1862 Validation Accuracy: 0.777344
Epoch  2, Batch  63 -Loss:  2115.6914 Validation Accuracy: 0.785156
Epoch  2, Batch  64 -Loss:  1087.1007 Validation Accuracy: 0.785156
Epoch  2, Batch  65 -Loss:  1407.6392 Validation Accuracy: 0.777344
Epoch  2, Batch  66 -Loss:  1131.4611 Validation Accuracy: 0.777344
Epoch  2, Batch  67 -Loss:  1121.2290 Validation Accuracy: 0.777344
Epoch  2, Batch  68 -Loss:   972.0236 Validation Accuracy: 0.777344
Epoch  2, Batch  69 -Loss:  1651.5623 Validation Accuracy: 0.773438
Epoch  2, Batch  70 -Loss:  1194.7233 Validation

...

Epoch  9, Batch 320 -Loss:   228.8063 Validation Accuracy: 0.839844
Epoch  9, Batch 321 -Loss:   241.5727 Validation Accuracy: 0.839844
Epoch  9, Batch 322 -Loss:   374.6034 Validation Accuracy: 0.835938
Epoch  9, Batch 323 -Loss:   225.7542 Validation Accuracy: 0.835938
Epoch  9, Batch 324 -Loss:   297.2312 Validation Accuracy: 0.835938
Epoch  9, Batch 325 -Loss:   223.0497 Validation Accuracy: 0.835938
Epoch  9, Batch 326 -Loss:   175.3218 Validation Accuracy: 0.835938
Epoch  9, Batch 327 -Loss:   317.5046 Validation Accuracy: 0.835938
Epoch  9, Batch 328 -Loss:   465.6767 Validation Accuracy: 0.835938
Epoch  9, Batch 329 -Loss:   471.5590 Validation Accuracy: 0.835938
Epoch  9, Batch 330 -Loss:   300.1049 Validation Accuracy: 0.835938
Epoch  9, Batch 331 -Loss:   380.4765 Validation Accuracy: 0.839844
Epoch  9, Batch 332 -Loss:   326.1864 Validation Accuracy: 0.839844
Epoch  9, Batch 333 -Loss:   288.3778 Validation Accuracy: 0.839844
Epoch  9, Batch 334 -Loss:   265.2426 Validation

Epoch 10, Batch  12 -Loss:   216.9241 Validation Accuracy: 0.851562
Epoch 10, Batch  13 -Loss:   231.4578 Validation Accuracy: 0.851562
Epoch 10, Batch  14 -Loss:   216.9965 Validation Accuracy: 0.851562
Epoch 10, Batch  15 -Loss:   362.3583 Validation Accuracy: 0.851562
Epoch 10, Batch  16 -Loss:   259.4800 Validation Accuracy: 0.851562
Epoch 10, Batch  17 -Loss:   170.7679 Validation Accuracy: 0.851562
Epoch 10, Batch  18 -Loss:   252.4858 Validation Accuracy: 0.851562
Epoch 10, Batch  19 -Loss:   222.6423 Validation Accuracy: 0.851562
Epoch 10, Batch  20 -Loss:   270.8563 Validation Accuracy: 0.847656
Epoch 10, Batch  21 -Loss:   333.4035 Validation Accuracy: 0.855469
Epoch 10, Batch  22 -Loss:   363.5435 Validation Accuracy: 0.855469
Epoch 10, Batch  23 -Loss:   327.0393 Validation Accuracy: 0.855469
Epoch 10, Batch  24 -Loss:   241.1222 Validation Accuracy: 0.855469
Epoch 10, Batch  25 -Loss:   415.0068 Validation Accuracy: 0.855469
Epoch 10, Batch  26 -Loss:   287.4537 Validation

Epoch 10, Batch 133 -Loss:   262.9792 Validation Accuracy: 0.847656
Epoch 10, Batch 134 -Loss:   271.9302 Validation Accuracy: 0.847656
Epoch 10, Batch 135 -Loss:   296.4564 Validation Accuracy: 0.851562
Epoch 10, Batch 136 -Loss:   375.7165 Validation Accuracy: 0.851562
Epoch 10, Batch 137 -Loss:   261.7928 Validation Accuracy: 0.851562
Epoch 10, Batch 138 -Loss:   319.1356 Validation Accuracy: 0.851562
Epoch 10, Batch 139 -Loss:   332.4482 Validation Accuracy: 0.847656
Epoch 10, Batch 140 -Loss:   192.5240 Validation Accuracy: 0.851562
Epoch 10, Batch 141 -Loss:   279.3231 Validation Accuracy: 0.847656
Epoch 10, Batch 142 -Loss:   230.8142 Validation Accuracy: 0.843750
Epoch 10, Batch 143 -Loss:   213.5919 Validation Accuracy: 0.843750
Epoch 10, Batch 144 -Loss:   259.7839 Validation Accuracy: 0.847656
Epoch 10, Batch 145 -Loss:   278.5977 Validation Accuracy: 0.847656
Epoch 10, Batch 146 -Loss:   297.9532 Validation Accuracy: 0.843750
Epoch 10, Batch 147 -Loss:   424.1591 Validation

Epoch 10, Batch 254 -Loss:   267.2916 Validation Accuracy: 0.859375
Epoch 10, Batch 255 -Loss:   265.0229 Validation Accuracy: 0.859375
Epoch 10, Batch 256 -Loss:   180.3090 Validation Accuracy: 0.859375
Epoch 10, Batch 257 -Loss:   194.3971 Validation Accuracy: 0.859375
Epoch 10, Batch 258 -Loss:   157.5112 Validation Accuracy: 0.855469
Epoch 10, Batch 259 -Loss:   251.4740 Validation Accuracy: 0.859375
Epoch 10, Batch 260 -Loss:   289.3602 Validation Accuracy: 0.859375
Epoch 10, Batch 261 -Loss:   248.1587 Validation Accuracy: 0.855469
Epoch 10, Batch 262 -Loss:   287.8627 Validation Accuracy: 0.859375
Epoch 10, Batch 263 -Loss:   213.5869 Validation Accuracy: 0.855469
Epoch 10, Batch 264 -Loss:   213.3196 Validation Accuracy: 0.859375
Epoch 10, Batch 265 -Loss:   295.2351 Validation Accuracy: 0.859375
Epoch 10, Batch 266 -Loss:   255.0199 Validation Accuracy: 0.859375
Epoch 10, Batch 267 -Loss:   284.6293 Validation Accuracy: 0.851562
Epoch 10, Batch 268 -Loss:   260.3082 Validation

Epoch 10, Batch 375 -Loss:   221.6217 Validation Accuracy: 0.843750
Epoch 10, Batch 376 -Loss:   275.6938 Validation Accuracy: 0.843750
Epoch 10, Batch 377 -Loss:   313.9822 Validation Accuracy: 0.847656
Epoch 10, Batch 378 -Loss:   253.4148 Validation Accuracy: 0.847656
Epoch 10, Batch 379 -Loss:   392.0220 Validation Accuracy: 0.847656
Epoch 10, Batch 380 -Loss:   187.4919 Validation Accuracy: 0.847656
Epoch 10, Batch 381 -Loss:   376.5320 Validation Accuracy: 0.847656
Epoch 10, Batch 382 -Loss:   147.6633 Validation Accuracy: 0.851562
Epoch 10, Batch 383 -Loss:   308.9879 Validation Accuracy: 0.847656
Epoch 10, Batch 384 -Loss:   334.9438 Validation Accuracy: 0.847656
Epoch 10, Batch 385 -Loss:   284.7655 Validation Accuracy: 0.847656
Epoch 10, Batch 386 -Loss:   293.8892 Validation Accuracy: 0.847656
Epoch 10, Batch 387 -Loss:   267.5585 Validation Accuracy: 0.847656
Epoch 10, Batch 388 -Loss:   326.8604 Validation Accuracy: 0.847656
Epoch 10, Batch 389 -Loss:   275.0373 Validation